In [16]:
#Qiskit modules
import qiskit
from qiskit import QuantumRegister as Q_R
from qiskit import ClassicalRegister as C_R
from qiskit_aer import Aer
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

#math modules
import math
import numpy as np
# SciPy minimizer routine
from scipy.optimize import minimize

#My libraries
import vqe_funcs


In [17]:
PI = np.pi

## Here are the functions performing Jordan-Wigner transformation for fermionc Pauli operators and kinetci energy of electrons 

Important notes:

1) This is spin-depenedent version, so each node has spin-coordinate and is described by 2 qubits.

   Spin-up state for node i is described by qubit i.

   Spin-down state for node i is described by qubit i + N, where N is the nodes number.

In [7]:


def sigma_x(node, nodes_number):
    if node > nodes_number - 1:
        print('Error: node index (''node'') cannot be higher than the total number of nodes')
        return -1
    interaction_string_1 = ''
    #term one
    if node > 0:
        for i in range(0, node):
            interaction_string_1 = interaction_string_1 + 'I'
    interaction_string_1 = interaction_string_1 + 'X'
    for i in range(node + 1, node + nodes_number):
        interaction_string_1 = interaction_string_1 + 'Z'
    interaction_string_1 = interaction_string_1 + 'Y'
    for i in range(node + nodes_number + 1, nodes_number * 2):
        interaction_string_1 = interaction_string_1 + 'I'
    #term two
    interaction_string_2 = ''
    if node > 0:
        for i in range(0, node):
            interaction_string_2 = interaction_string_2 + 'I'
    interaction_string_2 = interaction_string_2 + 'Y'
    for i in range(node + 1, node + nodes_number):
        interaction_string_2 = interaction_string_2 + 'Z'
    interaction_string_2 = interaction_string_2 + 'X'
    for i in range(node + nodes_number + 1, nodes_number * 2):
        interaction_string_2 = interaction_string_2 + 'I'
    bits = range(nodes_number * 2)
    interactions = [(interaction_string_1, bits, 1j/2)]
    interactions.append((interaction_string_2, bits, 1j/2))
    hamiltonian = SparsePauliOp.from_sparse_list(interactions, num_qubits = 2 * nodes_number)
    return hamiltonian

def sigma_y(node, nodes_number):
    if node > nodes_number - 1:
        print('Error: node index (''node'') cannot be higher than the total number of nodes')
        return -1
    interaction_string_1 = ''
    #term one
    if node > 0:
        for i in range(0, node):
            interaction_string_1 = interaction_string_1 + 'I'
    interaction_string_1 = interaction_string_1 + 'X'
    for i in range(node + 1, node + nodes_number):
        interaction_string_1 = interaction_string_1 + 'Z'
    interaction_string_1 = interaction_string_1 + 'X'
    for i in range(node + nodes_number + 1, nodes_number * 2):
        interaction_string_1 = interaction_string_1 + 'I'
    #term two
    interaction_string_2 = ''
    if node > 0:
        for i in range(0, node):
            interaction_string_2 = interaction_string_2 + 'I'
    interaction_string_2 = interaction_string_2 + 'Y'
    for i in range(node + 1, node + nodes_number):
        interaction_string_2 = interaction_string_2 + 'Z'
    interaction_string_2 = interaction_string_2 + 'Y'
    for i in range(node + nodes_number + 1, nodes_number * 2):
        interaction_string_2 = interaction_string_2 + 'I'
    bits = range(nodes_number * 2)
    interactions = [(interaction_string_1, bits, -1j/2)]
    interactions.append((interaction_string_2, bits, -1j/2))
    hamiltonian = SparsePauliOp.from_sparse_list(interactions, num_qubits = 2 * nodes_number)
    return hamiltonian

def sigma_z(node, nodes_number):
    if node > nodes_number - 1:
        print('Error: node index (''node'') cannot be higher than the total number of nodes')
        return -1

    #term two
    interaction_string_2 = ''
    if node > 0:
        for i in range(0, node):
            interaction_string_2 = interaction_string_2 + 'I'
    interaction_string_2 = interaction_string_2 + 'Z'
    for i in range(node + 1, 2 * nodes_number):
        interaction_string_2 = interaction_string_2 + 'I'

    #term four
    interaction_string_4 = ''
    for i in range(0, node + nodes_number):
        interaction_string_4 = interaction_string_4 + 'I'
    interaction_string_4 = interaction_string_4 + 'Z'
    for i in range(node + nodes_number + 1, 2 * nodes_number):
        interaction_string_4 = interaction_string_4 + 'I'

    
    bits = range(nodes_number * 2)
    interactions = [(interaction_string_2, bits, -1/2)]
    interactions.append((interaction_string_4, bits, 1/2))
    hamiltonian = SparsePauliOp.from_sparse_list(interactions, num_qubits = 2 * nodes_number)
    return hamiltonian


def kinetic_energy(t, nodes_number, periodic = True):
    interactions = []
    hamiltonian = []
    bits = range(nodes_number * 2)
    for i_node in range(0, nodes_number - 1):
        interaction_string_1 = ''
        for i in range(i_node):
            interaction_string_1 = interaction_string_1 + 'I'
        interaction_string_1 = interaction_string_1 + 'XX'
        #print(interaction_string_1)
        interaction_string_2 = ''
        for i in range(i_node):
            interaction_string_2 = interaction_string_2 + 'I'
        interaction_string_2 = interaction_string_2 + 'YY'
        
        interaction_string_3 = ''
        for i in range(i_node + nodes_number):
            interaction_string_3 = interaction_string_3 + 'I'
        interaction_string_3 = interaction_string_3 + 'XX'
        
        interaction_string_4 = ''
        for i in range(i_node + nodes_number):
            interaction_string_4 = interaction_string_4 + 'I'
        interaction_string_4 = interaction_string_4 + 'YY'
        interactions.append((interaction_string_1, bits, -t/2))
        interactions.append((interaction_string_2, bits, -t/2))
        interactions.append((interaction_string_3, bits, -t/2))
        interactions.append((interaction_string_4, bits, -t/2))
    
    
        
    if periodic == True:
        interaction_string_1 = ''
        interaction_string_1 = interaction_string_1 + 'X'
        for i in range(1, nodes_number - 1):
            interaction_string_1 = interaction_string_1 + 'I'
        interaction_string_1 = interaction_string_1 + 'X'
        
        interaction_string_2 = ''
        interaction_string_2 = interaction_string_2 + 'Y'
        for i in range(1, nodes_number - 1):
            interaction_string_2 = interaction_string_2 + 'I'
        interaction_string_2 = interaction_string_2 + 'Y'
        
        interaction_string_3 = ''
        for i in range(0, nodes_number):
            interaction_string_3 = interaction_string_3 + 'I'
        interaction_string_3 = interaction_string_3 + 'X'
        for i in range(nodes_number + 1, 2 * nodes_number - 1):
            interaction_string_3 = interaction_string_3 + 'I'
        interaction_string_3 = interaction_string_3 + 'X'
        
        interaction_string_4 = ''
        for i in range(0, nodes_number):
            interaction_string_4 = interaction_string_4 + 'I'
        interaction_string_4 = interaction_string_4 + 'Y'
        for i in range(nodes_number + 1, 2 * nodes_number - 1):
            interaction_string_4 = interaction_string_4 + 'I'
        interaction_string_4 = interaction_string_4 + 'Y'
        
        interactions.append((interaction_string_1, bits, -t/2))
        interactions.append((interaction_string_2, bits, -t/2))
        interactions.append((interaction_string_3, bits, -t/2))
        interactions.append((interaction_string_4, bits, -t/2))

    hamiltonian =  (SparsePauliOp.from_sparse_list(interactions, num_qubits = 2 * nodes_number))
    return hamiltonian




## Magnetization distribution description

In [5]:
#Uniform magnetization along z axis
nodes_number = 4
mag = []

def uniform_z(nodes_number):
    mag = []
    for i in range(nodes_number):
        mag.append([0, 0, 1])
    return mag

## Full Hamiltonian

The Hamiltonian includes kinteic energy and s-d exchage energy

In [9]:
def full_ham(nodes_number, magnetization, J, t, periodic = True):
    hamiltonian = []
    hamiltonian = kinetic_energy(t, nodes_number, periodic)
    #mag_ham = SparsePauliOp.from_sparse_list([], num_qubits = 2 * nodes_number)
    #mag_ham = sum([], J * magnetization[1][2] * sigma_x(1, nodes_number))
    mag_ham = []
    for node in range(nodes_number):
        if magnetization[node][0] != 0:
            #mag_ham.append(J*magnetization[node][0] * sigma_x(node, nodes_number))
            mag_ham = sum(mag_ham, J * magnetization[node][0] * sigma_x(node, nodes_number))
        if magnetization[node][1] != 0:
            #mag_ham.append(J*magnetization[node][1] * sigma_y(node, nodes_number))
            mag_ham = sum(mag_ham, J * magnetization[node][1] * sigma_y(node, nodes_number))
        if magnetization[node][2] != 0:
            #mag_ham.append(J*magnetization[node][2] * sigma_z(node, nodes_number))
            mag_ham = sum(mag_ham, J * magnetization[node][2] * sigma_z(node, nodes_number))
    hamiltonian = sum(mag_ham, hamiltonian)
    return hamiltonian

nodes_number = 2
mag = uniform_z(nodes_number)
full_ham(nodes_number, mag, 4, 1, periodic = True)

SparsePauliOp(['IIXX', 'IIYY', 'XXII', 'YYII', 'IIXX', 'IIYY', 'XXII', 'YYII', 'IIZI', 'ZIII', 'IIIZ', 'IZII'],
              coeffs=[-0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j, -0.5+0.j,
 -0.5+0.j, -2. +0.j,  2. +0.j, -2. +0.j,  2. +0.j])

## Creating an anzatz

In [25]:
def anzatz_qc(theta, nodes_number, electrons_number):
    qubits_number = 2 * nodes_number
    q_r = Q_R(qubits_number)
    v_qc = QuantumCircuit(q_r)
    for i in range(electrons_number):
        v_qc.x(i)
    v_qc = vqe_funcs.exc(qubits_number, theta, v_qc)
    return v_qc

In [76]:
nodes_number = 2
electrons_number = 2
n_qubits = nodes_number * 2
len_se = n_qubits * (n_qubits - 1) / 2
len_de = n_qubits * (n_qubits - 1) * (n_qubits - 2) * (n_qubits - 3) / 4 / 2
len_tot = int(len_se + len_de)

theta = [0]
for i in range(len_tot):
    theta.append(0)

theta[8] = PI / 9

qc = anzatz_qc(theta, nodes_number, electrons_number)

#qc.draw('mpl')


In [62]:
qc.decompose().count_ops()

OrderedDict([('cx', 184), ('u2', 120), ('r', 120), ('u1', 36), ('u3', 2)])

In [27]:
# defining a cost function
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
def estim(theta, nodes_number, hamiltonian, electrons_number):
    v_qc = anzatz_qc(theta, nodes_number, electrons_number)
    
    estimator = StatevectorEstimator()
    job = estimator.run([(v_qc, hamiltonian)])
    estimator_expvals = job.result()[0].data.evs
    
      
    #print 
    #print('Energy: ' + str(estimator_expvals))
    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = theta
    cost_history_dict["cost_history"].append(estimator_expvals)
    return estimator_expvals

In [78]:
# Hamiltonian parameters
nodes_number = 3 #number of nodes
n_qubits = nodes_number * 2
J = -5 #s-d exchange constant
t = 2 #hopping matrix element (kinetic energy coefficient)
electrons_number = 3

#Cost history initialization
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

import random
num = random.random()
#print(num)


mag = []
mag = uniform_z(nodes_number)

#initialize the state
len_se = n_qubits * (n_qubits - 1) / 2
len_de = n_qubits * (n_qubits - 1) * (n_qubits - 2) * (n_qubits - 3) / 4 / 2
len_tot = int(len_se + len_de)
x0 = []
for i in range(len_tot):
    x0.append(random.random())

#define the Hamiltonian
hamiltonian = full_ham(nodes_number, mag, J, t, periodic = False)
#print(hamiltonian)
#Optimization
res = minimize(
        estim,
        x0,
        args=(nodes_number , hamiltonian, electrons_number),
        method="cobyla",
    )


In [69]:
q_r = Q_R(nodes_number * 2)
cl_r = C_R(nodes_number * 2)
qc_f = QuantumCircuit(q_r,cl_r)
qc_1 = anzatz_qc(res.x, nodes_number, electrons_number)
qc_f.append(qc_1, q_r)

#for i in range(bit_size):
#    qc_f.measure(i,i)
SimulatorAer = AerSimulator()

qc_f.save_statevector()

#qc_f.decompose().draw('mpl')

In [70]:
qc_f.decompose().count_ops()

OrderedDict([('cx', 184),
             ('h', 120),
             ('rx', 120),
             ('rz', 36),
             ('x', 2),
             ('save_statevector', 1)])

In [71]:
from qiskit.quantum_info import partial_trace, DensityMatrix
from qiskit.visualization import plot_state_city

circ = transpile(qc_f, backend = SimulatorAer)
result = SimulatorAer.run(circ,shots = 1).result()
ground_state = result.get_statevector(circ)

In [72]:
import sys
sys.path.insert(0, 'C:/Users/Oleg/Google Диск/QC/Codes/QC-qiskit-codes/Library')
import aux_func as af
n_nonzero = 0
n_states = pow(2, 2 * nodes_number)
states = []
prob = []
for i in range(n_states):
    pr = pow(abs(ground_state[i]), 2)
    if pr>1/n_states:
        n_nonzero = n_nonzero + 1
        states.append(i)
        prob.append(pr)
        print('state: ' + str(af.int_2_bin_word(i, 2 * nodes_number)) + ', prob: ' + str(pr) + ', complex amplitude: ' + str(ground_state[i]))
    
print('Energy: ' + str(res.fun))

state: 0011, prob: 0.9999969422162966, complex amplitude: (0.9999984711069796+3.4771682655859982e-15j)
Energy: -9.999980905051217
